###### ---
# Unit16: Ambient Noise Tomography

This notebook has the activities of the Course **ProSeisSN**. It deals with time series processing using a passive seismic dataset using [ObsPy](https://docs.obspy.org/).

#### Dependencies: Obspy, Numpy, Matplotlib
#### Reset the Jupyter notebook in order to run it again, press:
***Kernel*** -> ***Restart & Clear Output***

In [ ]:
"""
====================== Leads to Colab ======================
!git clone https://github.com/jandyr/ProSeisSN
!cd ProSeisSN
"""
#!pip install pycwt
#import pycwt
!pip install disba mpi_master_slave
!pip install pylops
#!pip install obspy
!pip install import-ipynb
import import_ipynb

In [ ]:
#------ Import OS Libraries
import sys
import os
import copy
#------ Work with the directory structure to include auxiliary codes
print('\n Local directory ==> ', os.getcwd())
print('  - Contents: ', os.listdir(), '\n')

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path+"/CodePy")

#%run ../CodePy/ImpMod.ipynb
%run ../CodePy/ImpMod
#------ Alter default matplotlib rcParams
from matplotlib import rcParams
import matplotlib.dates as dates
# Change the defaults of the runtime configuration settings in the global variable matplotlib.rcParams
plt.rcParams['figure.figsize'] = 9, 5
#plt.rcParams['lines.linewidth'] = 0.5
plt.rcParams["figure.subplot.hspace"] = (.9)
plt.rcParams['figure.dpi'] = 100
#------ Magic commands
%matplotlib inline
%matplotlib widget
#%pylab notebook
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

---
## Use data files from TTB24
- **The Receiver Array**
The array has 4 3-C geophones hooked to a $L=69$m cable, using takeouts spaced $\delta l=6$m from each other. The GS-20DX geophones have a natural frequency of $f_n=4.5\textrm{Hz}$. The array has a irregular cross shape.
- **The data**

<div style="text-align: center;">
<img src="./ttb24T.png" width="700">
</div>

In [ ]:
"""
====================== READ PHONES LOCATIONS ======================
"""
#------ Read the phones cartesian locations
#--- Reads the CSV file ─> phone# (int), x, y (floats)
ttb24_loc = u.RGloc('../Data/'+'ttb24_loc.dat')
#------ Read the phones geographic locations
#--- Reads the CSV file with (lat,lon) in degress locations
ttb24_gloc = u.RGloc('../Data/'+'ttb24_gloc.dat')
#
#------ Plot gather in cartesian
p.pgather(ttb24_loc[:,1], ttb24_loc[:,2], ttb24_loc[:,0], coord='cartesian')

In [ ]:
def creastrm(st, gloc):
#------ Initialization
    gather = Stream()
    idx = [ [0 , 2], [3 , 5], [6 , 7], [8 , 10] ]
    ch  = ['BHZ', 'BH1', 'BH2']
    dist= [0., 70.411, 27.151, 54.074]
#------ Loop through traces
    for i in range(len(idx)):
        k = 0
#   row index, 1st column of idx| row index, 2nd column of idx
        for j in range(idx[i][0], idx[i][1]+1):
            tr = Trace(data=st[j].data)
            tr.stats.sampling_rate = st[j].stats.sampling_rate
            tr.stats.starttime     = st[j].stats.starttime
            tr.stats.network       = 'TTB24'
            tr.stats.station       = f"{str(i+1)}"
            tr.stats.channel       = ch[k]
            tr.stats.location      = "SMT"
            tr.stats.distance      = dist[i]             # Distance btw contiguous phones
            tr.stats.coordinates = \
                                 AttribDict({'latitude':  gloc[i,1],
                                             'longitude': gloc[i,2],
                                             'elevation': 10.})
            print(f">> Working with phone {i+1} and trace {ch[k]}.")
            k = k + 1
#
            gather += tr                                 # gather.append(tr)
#
    print(f"\n>> A new {len(gather)}-trace gather is created.")
#------ Return gather
    return gather
"""
====================== READ THE SEISMIC DATA LOCALLY ======================
"""
import random
#------ Read the seismic data
filename = ['S20240519_045005483', 'S20240519_105307548']
#
print(f'\nFiles: {filename}')
#ent = random.choice(filename)
ent = input(f'   Enter a file index 1 or 2 , rtn={1}\n') or '1'
ent = ent.rstrip().split(' ')[0]
print(f">> Read data file {filename[int(ent)-1]}")
ent = '../Data/ttb24/'+filename[int(ent)-1]+'.seg2'
#
#------- Read the data file as a SEG2 object.
st     = read(ent)
#
#------- Print stream information
#dummy = float(st[-1].stats.seg2.RECEIVER_LOCATION)
print(f">> Gather acquired on {st[0].stats.starttime}, has {int(st[0].stats.npts)} data points.")
"""
================= Create a new stream from the SEG2 stream ======================
                         Retain a gather copy
"""
#------ Create a new stream from the SEG2 stream.
#       1) Adds coordinates to gather. Stores a copy in gather0
#       2) Gather baricenter = bcenter.
gather = creastrm(st, ttb24_gloc)
gather0 = gather.copy()
#
#--- Phone choice
phone = None
gather.plot()

---
### Data processing

In [ ]:
#
"""
================= Filter data and look at the frequency contents ======================
                    Create a new stream from the SEG2 stream
"""
#
#------- Remove mean and trend + filter the stream
#--- Filter parameters: change them as you wish.
MTparam = [ 1,   1,    'bp',  4.5,   40.,   1,    0]
# └─────> [dtr, line, ftype, Fmin, Fmax, taper, gain]
#                                          └─> data will be also windowed at trace normalization and spectral whitening
ent = str(MTparam[3]) + ' ' + str(MTparam[4])
ent = input(f'\n>> Enter filter min and max frequencies (dflt = {MTparam[3]}, {MTparam[4]})') or ent
ent = ent.rstrip().split(' ')
MTparam[3], MTparam[4] = [float(dummy) for dummy in ent]
#
#------- Process
gather.detrend("linear")
gather.detrend("demean")
#
gather, ftype, flims = u.TrFlt(gather, ent=['bs', 59.2, 60.8])
print(f">> Notched original trace btw 59.2 and 60.8Hz")
#
gather, ftype, flims = u.TrFlt(gather, ent = [MTparam[2], MTparam[3], MTparam[4]])
print(f">> Useful range due to {ftype} filter: {flims[0]} to {flims[1]}Hz.")

#---  Taper the data with 2.5% Hanning
gather.taper(type = 'hann', max_percentage = 0.025)
#
#------- Check frequency contents to accept preprocessing
#--- Pick up a random phone/trace
phone = phone if phone is not None else np.random.randint(1, len(gather)+1)
print(f' Random phone {phone} ')
#--- Go to trace instead of phone: trace = phone -1
phone = phone - 1
#--- Relative time: nummpy array
time = gather[phone].times(type="relative")
#         Clear last plot -> plt.clf()
#--- Plot Trace+Spectrogram
p.Pspect(time, gather[phone])
#
#------- Once filtering is accepted create a new backup for gather
ent = input(f' Run this cell again (rtn= No, else plot Spectrogram)?: ') or False
if not ent:
    gather0 = gather.copy()
    print(f' A new stream backup was created.')
else:
    gather = gather0.copy()

---
### Choose an event
- Hint: file S20240519_045005483 - phone 1 - [372, 374]s, [401.5-402.7]

In [ ]:
print(gather)
#
#------ plot
dummy = gather[0].times(type="relative")             # time axis (s)
fNy = gather[0].stats.sampling_rate / 2.             # Nyquist
for i in range(len(gather)):
    ch = gather[i].stats.channel
    FtrTplt = np.fft.rfft(gather[i].data)[1:]        # Discard DC component
    if i == 0: freq = np.linspace(1, fNy, len(FtrTplt))   # Freq. axis (HZ). Discard f=0
    p.pltTrSp( dummy, gather[i].data, freq, abs(FtrTplt),
              x1label='s', y1label='Ampl.', y1log=False, clr1 = 'k', 
              x2label='s', y2label=ch, y2log=True, clr2 = 'r' )

### Clip the data to an event
1) file S20240519_045005483 - phone 1 - [372, 374]s, [401.5-402.7]

In [ ]:
import copy
"""
================= Clip the data ======================
"""
ent = input(f'   Enter a phone [1-4] , rtn={1}\n') or '1'
ent = int(ent.rstrip().split(' ')[0])
if ent == 3: raise ValueError("Phone 3 has only 2 components.")
#--- Create an independent copy
phone = copy.deepcopy(ent)
ent = ent - 1
#
#------ Create a data Stream object with the 3-components
idx = [ [0 , 2], [3 , 5], [6 , 7], [8 , 10] ]
st = []
#--- Elements of the i-th row of idx containing the components
for j in range(idx[ent][0], idx[ent][1]+1):
    st.append( gather[ j ] )
#--- New data Stream
st = Stream(traces=st)
#
#------ Cut the trace to a window and taper it: 5% hann at both sides
ent = input(f' Enter t0 and t1 to cut: ')
ent = ent.rstrip().split(' ')
t0 = float(ent[0])
t1 = float(ent[1])
#
for tr in st:
    tr.trim(starttime=tr.stats.starttime + t0, endtime=tr.stats.starttime + t1)
    tr.taper(0.025)
#
print(f'Trimmed stream: {st}')
#
#st.plot(color='b')


In [ ]:
"""
================= Clip the original gather z-components ======================
"""
beam = []
for i in idx:
    beam.append( gather0[ i[0] ] )
beam = Stream(traces=beam)
for tr in beam:
    tr.trim(starttime=tr.stats.starttime + t0, endtime=tr.stats.starttime + t1)
    tr.taper(0.025)
#
print(beam)
beam.plot(type="relative")

In [ ]:
"""
====================== BEAMFORMING ======================
"""
dummy = UTCDateTime()
# ---------- Use FK Analysis
out, stime, etime = u.BeamFK(beam, [MTparam[3], MTparam[4]], ttb24_gloc)
print("\n>> Total time in Beamforming: %f\n" % (UTCDateTime() - dummy))
#---------- Change output
t, rel_power, abs_power, baz, slow = out.T
#--- Time
T = np.linspace(stime, etime, num=len(out))
#--- Semblance -> Fisher
F = (len(gather)-1) * out[:, 1] / (1 - out[:, 1])
#--- FK power
FKp = out[:, 2]
#--- baz
#out[:, 3] = out[:, 3] % 360.
baz = baz % 360.
#--- Slowness -> Velocity
V = 1.e3 / out[:,4]
#------------- print
sys.stdout.write('\n')
print(f'\n>>  t   Fisher  FKpwr   baz(deg) vel(m/s)')    
for i in range(len(out)):
    print(f'   {round(T[i],2)}, {round(F[i],2)}, {round(FKp[i],2)}, {round(baz[i],2)}, {round(V[i],2)}')    
#------------ Plot
p.pltbaz(out, stime , etime)

In [ ]:
"""
================= Plot clipped data and their spectra ======================
"""
#
#------ plot
dummy = st[0].times(type="relative")         # time axis (s)
fNy = st[0].stats.sampling_rate / 2.         # Nyquist
for i in range(len(st)):
    ch = gather[i].stats.channel
    FtrTplt = np.fft.rfft(st[i].data)[1:]        # Discard DC component
    if i == 0: freq = np.linspace(1, fNy, len(FtrTplt))   # Freq. axis (HZ). Discard f=0
    p.pltTrSp( dummy, st[i].data, freq, abs(FtrTplt),
              x1label='s', y1label='Ampl.', y1log=False, clr1 = 'k', 
              x2label='s', y2label=ch, y2log=True, clr2 = 'r' )
plt.show()

In [ ]:
"""
    Rotates horizontal components of a seismogram
    <Parameters>
    n  -> Data of the North component of the seismogram (Y-axis)
    e  -> Data of the East component of the seismogram  (X-axis)
    a  -> Angle (degrees) from Z, a > 0 is a clockwise rotation from N.
    <Returns>
    t -> Transversal component of seismogram (X-axis).
    r -> Radial component of seismogram (Y-axis).
"""

def rotate(e, n, a):
    t = np.cos(np.radians(a)) * e - np.sin(np.radians(a)) * n
    r = np.sin(np.radians(a)) * e + np.cos(np.radians(a)) * n
    return t, r
#
"""
================= Plot clipped data and their spectra ======================
"""
alpha = np.arange(0.,360.,2.)
tvall = []
#
#------ Rotate BH1 and BH2 components and estimate transverse energy Te
#--- Angle a is relative to orientation of channels BH1 and BH2.
for a in alpha:
    hhT, hhR = rotate(st[2].data,st[1].data, a)
    Te  = np.dot(hhT,hhT)
    tvall.append(Te)
#
tval = np.array(tvall)
tval = tval / np.max(np.abs(tval))
mina = alpha[np.argmin(tval)]
print(f'\n>> Optimal angle at phone {phone} = {mina}, or {mina-180}')
thetat = str(mina); thetalt = str(mina-180)
#
#------ plot T energy vs. rotation angle:
plt.figure(figsize=(7, 3.5))
plt.plot(alpha, tval, 'r-', linewidth=2)
plt.xlabel('Angle'+r'($\theta$)', fontsize=10)
plt.ylabel(r'E($\theta$)', fontsize = 10)
# Add a vertical line at the point of minimum energy
#plt.plot([mina, mina],[0, .1],linewidth = 2)
plt.text(mina+2,.03,r'$\theta_t$ = '+ thetat +r'$\degree$', fontsize=10)
plt.show()


<div style="text-align: center;">
<img src="./ttb24g.png" width="400">
</div>

### Rotate BH1 and BH2 components
1) The correct horizontal orientation of a sensor can be deduced from the measured polarization azimuth of an incoming P
wave or surface wave.

2) The angles
$$\theta_{0}=\theta_{T}-\theta_{M}$$
represent the **BH1 azimuth, source azimuth, and baz** respectively.

4) 

Rayleigh waves (or so called ground roll) move both longitudinally and horizontally and result in elliptical movement, similar to the rolling water waves but against the direction of propagation.

Love waves move horizontally and at the right angles to the direction of propagation, but cannot propagate in water like S-waves.

Rotate BH1 and BH2 components to find the rotation angle in `alpha` that miminizes P-wave energy on one of the two channels. 
Minimize the transverse energy following rotation to the correct radial direction


Love waves are horizontally polarized shear waves ravelling along the Earth's surface. They are generated by the interaction of seismic shear waves with the Earth's surface. Rayleigh waves involve a combination of vertical and horizontal motion, causing elliptical particle motion. They are slower than Love waves because they involve more complex ground displacement.

Love waves typically travel at speeds comparable to or slightly slower than the S-wave velocity of the Earth's crust. Rayleigh waves generally travel at about 0.9 times the S-wave velocity.

<div style="text-align: center;">
<img src="./pprl.png" width="1000">
</div>

In [ ]:
"""
================= Rotate horizontal channels over the optimal angle ======================
"""
#
#------ Rotate to optimum angle mina
#                   |-- BH2 --|--- BH1 --|
trT, trR = rotate(st[2].data,st[1].data,mina)
#
#------ Create a rotated gather
stR = Stream()
for i, t in enumerate(st):
    if t.stats.channel == 'BHZ':
#--- Vertical component remains unchanged
        tr = Trace(data=t.data)
        tr.stats = t.stats
        stR += tr
#--- East component (X-axis) -> Transversal component
    elif t.stats.channel == 'BH1':
        tr = Trace(data=trT)
        tr.stats = t.stats
        tr.stats.channel = 'BHT'
        stR += tr
#--- North component (Y-axis) -> Radial component
    elif t.stats.channel == 'BH2':
        tr = Trace(data=trR)
        tr.stats = t.stats
        tr.stats.channel = 'BHR'
        stR += tr
    else:
        raise ValueError("Invalid component")
#
print(stR)
#
#------ Plot them
dummy = stR[0].times(type="relative")         # time axis (s)
fNy = stR[0].stats.sampling_rate / 2.         # Nyquist
for i in range(len(stR)):
    ch = gather[i].stats.channel
    FtrTplt = np.fft.rfft(stR[i].data)[1:]        # Discard DC component
    if i == 0: freq = np.linspace(1, fNy, len(FtrTplt))   # Freq. axis (HZ). Discard f=0
    p.pltTrSp( dummy, stR[i].data, freq, abs(FtrTplt),
              x1label='s', y1label='Ampl.', y1log=False, clr1 = 'k', 
              x2label='s', y2label=ch, y2log=True, clr2 = 'r' )
plt.show()

In [ ]:
# read in data from file, subtract the mean, set time axis array

tvec = stR[0].times(type="relative") 
#nLen = len(tvec) 
deltaT = tvec[1] - tvec[0]
trT = stR[1] - np.mean(stR[1])
trR = stR[2] - np.mean(stR[2])
traces = np.array([trT,trR])
#
plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.plot(tvec,traces[1],'r-',label='Radial')
plt.plot(tvec,traces[0],'b-',label='Transverse')
#plt.ylim([-8,8])
plt.legend(); plt.xlabel('Time (s)')
plt.title('R, T traces')
plt.subplot(122)
plt.scatter(traces[0],traces[1],c=tvec)
#plt.ylim([-8,8]); plt.xlim([-8,8])
plt.xlabel('Transverse'); plt.ylabel('Radial')
plt.axis('equal'); plt.title('Motion')
plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)
plt.show()